# SVM

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
features = pd.read_csv('time_features_Oz-Fpz/combined_features.csv',index_col=0)
features = features[~np.isnan(features['scores'])]
features = features[features.scores < 6]
features.head()
X = features[['std', 'skewness', 'ZCR', 'mean', 'min', 'max', 'kurtosis', '25%', '75%', '50%']]
y = features['scores']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

is_nan = np.any(np.isnan(y_train))
is_finite = np.all(np.isfinite(y_train))
print("Scores of value higher than intended: ", (y > 6).any())
print("Is nan should be false, Result: ", is_nan)
print("Is finite should be true, Result: ", is_finite)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Scores of value higher than intended:  False
Is nan should be false, Result:  False
Is finite should be true, Result:  True


In [2]:
from sklearn.preprocessing import MinMaxScaler
size = 1

Ac_array = np.zeros(size, )
Se_array = np.zeros((size, 6))
Sp_array = np.zeros((size, 6))
t_array = np.zeros(size, )
i_array = np.zeros(size, )

for i in range(size):
    print("Processing the "+str(i+1)+" iteration...")
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    X_train = scaling.transform(X_train)
    X_test = scaling.transform(X_test)
    start = time.time()
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    end = time.time()
    elapsed = end - start
#     print("Time to complete: ", elapsed, "seconds")

    predictions = clf.predict(X_test)

    accuracy = accuracy_score(y_test, predictions)
#     print("Accuracy: " , accuracy)
    CM = confusion_matrix(y_test, predictions)
    FP = CM.sum(axis=0) - np.diag(CM)  
    FN = CM.sum(axis=1) - np.diag(CM)
    TP = np.diag(CM)
    TN = CM.sum() - (FP + FN + TP)
    Sensitivity = TP/(TP+FN)
    Specificity = TN/(TN+FP)
    
    Ac_array[i] = accuracy
    Se_array[i] = Sensitivity
    Sp_array[i] = Specificity
    t_array[i] = elapsed
#     print("Specificity: ", Specificity.tolist())
#     print("Sensitivity: ", Sensitivity.tolist())
    print("Completed!")

Processing the 1 iteration...
Completed!


In [3]:
df = pd.DataFrame(Ac_array)
df.columns = ['Accuracy']
df1 = pd.DataFrame(t_array)
df1.columns = ['time_elapsed']
df2 = pd.DataFrame(Sp_array)
df2.columns=['Sp0','Sp1','Sp2','Sp3','Sp4','Sp5',]
df3 = pd.DataFrame(Se_array)
df3.columns=['Se0','Se1','Se2','Se3','Se4','Se5',]
result = pd.concat([df1, df, df3, df2], axis=1)
result.to_csv("Results_SVM/results_SVM_Oz-Fpz.csv")